# 프로젝트 - Movielens 영화 추천 실습

MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보자.<br>
활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터이다.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. `MovieLens 1M Dataset` 사용 권장.
* 별점 데이터는 대표적인 explicit 데이터이다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있다.
* 별점을 __시청횟수__로 해석해서 생각.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외.

`MovieLens 1M Dataset` 데이터셋을 아래와 같이 직접 다운 받고 압축해제 하기.

```bash
1) wget으로 데이터 다운로드
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

2) 다운받은 데이터를 작업디렉토리로 옮김
$ mv ml-1m.zip ~/aiffel/recommendata_iu/data

3) 작업디렉토리로 이동
$ cd ~/aiffel/recommendata_iu/data

4) 압축 해제
$ unzip ml-1m.zip
```

/home/aiffel-dj10/AIFFEL/Exploration/E8_

## 1) 데이터 준비와 전처리
---
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

### 필요한 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares

### 데이터 불러오기

In [2]:
rating_file_path=os.getenv('HOME') + '/AIFFEL/Exploration/E8_Recommender_system/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### rating 3점 이상만 남기기

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


### rating 컬럼 이름을 count로 바꾸기

In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [5]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

### 영화 제목을 보기 위해 movies 데이터 불러오기

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/AIFFEL/Exploration/E8_Recommender_system/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## 2) 분석해 봅시다.
---
* ratings에 있는 유니크한 영화 개수
* rating에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

### ratings에 있는 유니크한 영화 개수

In [7]:
print(ratings['movie_id'].nunique())

3628


### rating에 있는 유니크한 사용자 수

In [8]:
print(ratings['user_id'].nunique())

6039


### 가장 인기 있는 영화 30개(인기순)

In [9]:
mv_merge = pd.merge(ratings, movies, how='left')
mv_merge

mv_count = mv_merge.groupby('title')['user_id'].count()
mv_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
---
내가 선호하는 Jurassic Park (1993), Toy Story (1995), Toy Story 2 (1999), E.T. the Extra-Terrestrial (1982), Schindler's List (1993) 다섯가지를 추가한다.

In [10]:
my_favorite = ['Jurassic Park (1993)', 'Toy Story (1995)', 'Toy Story 2 (1999)', 'E.T. the Extra-Terrestrial (1982)', "Schindler's List (1993)"]
               
for movie in my_favorite:
    print(movies[movies['title'] == movie])
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

     movie_id                 title                    genre
476       480  Jurassic Park (1993)  Action|Adventure|Sci-Fi
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      movie_id               title                        genre
3045      3114  Toy Story 2 (1999)  Animation|Children's|Comedy
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
      movie_id                              title  \
1081      1097  E.T. the Extra-Terrestrial (1982)   

                                genre  
1081  Children's|Drama|Fantasy|Sci-Fi  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     movie_id                    title      genre
523       527  Schindler's List (1993)  Drama|War
~~~~~~~~~~~~~~~~~~~~~~~~~

### 데이터 프레임에 추가하기

In [11]:
my_movielist = pd.DataFrame({'user_id': ['isaac']*5, 'count':[5]*5, 'movie_id': [480, 1, 3114, 1097, 527]})

if not ratings.isin({'user_id':['zimin']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    ratings = ratings.append(my_movielist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

ratings.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,count,timestamp
1000203,6040,1090,3,956715518.0
1000205,6040,1094,5,956704887.0
1000206,6040,562,5,956704746.0
1000207,6040,1096,4,956715648.0
1000208,6040,1097,4,956715569.0
0,isaac,480,5,NaN
1,isaac,1,5,NaN
2,isaac,3114,5,NaN
3,isaac,1097,5,NaN
4,isaac,527,5,NaN


### 사용하는 칼럼만 남겨주기. (timestamp 제거)

In [12]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


### user_id와 movie_id를 indexing 하기.

In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,count
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,107,5
1,6039,40,5
2,6039,50,5
3,6039,26,5


## 4) CSR matrix를 직접 만들어 봅시다.
---

In [14]:
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
---


In [15]:
# implicit 라이브러리에서 권장하고 있는 부분입니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

### AlternatingLeastSquares 모델 선언

In [16]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=10, dtype=np.float32)

### nput으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해주기.

In [17]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

### 모델 훈련

In [18]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/10 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
---

### isaac이 처음에 추가한 Jurassic_Park의 선호도 구하기.

In [19]:
isaac, Jurassic_Park = user_to_idx['isaac'], movie_to_idx[480] #Jurassic_Park의 인덱스
isaac_vector, Jurassic_Park_vector = als_model.user_factors[isaac], als_model.item_factors[Jurassic_Park]

In [20]:
isaac_vector

array([-0.02386551, -0.87607867,  0.36801165, -0.29957303,  0.02882819,
       -0.6979108 , -0.17374676,  0.27483144,  0.5963833 ,  0.4019746 ,
        0.44503725, -0.49896365,  0.12812954,  0.7073384 , -0.7197418 ,
        0.47106862, -0.6381918 ,  0.5100366 ,  0.14978908,  0.7785976 ,
       -1.0330039 , -0.44802856, -0.7169243 ,  0.39942265, -0.40406215,
        0.90051967, -0.47214532, -0.84564555,  0.8989661 ,  0.91104245,
        1.2129846 ,  0.80364513, -0.5711532 ,  0.6333089 , -0.06375101,
       -0.16004841,  0.08671572,  0.7615351 , -0.97713834,  0.47015837,
       -0.14112319,  0.21637633,  1.0569081 , -0.22157267, -0.602088  ,
       -1.0190368 ,  0.14026661, -1.2144214 ,  0.4363405 , -0.14443816,
       -1.3019031 ,  0.5404452 , -0.5976917 ,  0.25918686, -0.05550312,
        0.3691377 , -0.30616477,  0.279329  ,  0.5445898 , -0.39094558,
       -0.02102244,  0.8141729 , -0.54683006, -0.52023536, -0.07848987,
       -0.55622315,  0.9794422 , -0.10185075, -0.03328773, -0.15

In [21]:
Jurassic_Park_vector

array([ 2.5728627e-03,  2.4104880e-02,  2.2717193e-03,  2.8791416e-02,
       -1.9117048e-02, -6.2603136e-03, -2.5445204e-02, -2.8015422e-02,
        1.2143780e-02,  2.8613903e-02, -9.2518756e-05, -1.8976922e-03,
        4.4220481e-03,  2.4245406e-02, -1.4281294e-03,  3.9138120e-02,
        1.3301191e-02, -1.0204745e-02, -1.0414334e-02,  2.0861251e-02,
        1.3740401e-03, -5.3600627e-03,  8.3918246e-03,  2.2508372e-02,
        8.3364835e-03,  1.3065014e-02, -3.7078720e-03, -2.6109401e-02,
        2.4294479e-02,  8.9228759e-03,  1.1113954e-02,  5.7978509e-03,
        8.9305183e-03,  8.2871392e-03, -1.0974451e-02,  1.3871003e-02,
        1.7008021e-02,  1.0997117e-02, -1.4930805e-02,  8.3667887e-03,
       -7.2567929e-03,  1.3588779e-03,  3.4346518e-03, -7.1670567e-03,
        1.0801674e-02, -2.0758195e-02, -8.5925328e-04, -2.9594749e-02,
        2.2954797e-02,  3.0657053e-02,  2.2746516e-02,  3.5363201e-02,
        7.3435362e-03,  2.7277721e-02,  3.3053007e-02, -1.4730510e-02,
      

In [22]:
# isaac과 Jurassic_Park를 내적하는 코드
np.dot(isaac_vector, Jurassic_Park_vector)

0.43541944

### 그 외의 영화 Jumanji (1995) 의 선호도 구하기.

In [23]:
Jumanji = movie_to_idx[2] #Jumanji (1995)의 인덱스
Jumanji_vector = als_model.item_factors[Jumanji]

In [24]:
Jumanji_vector

array([-0.00156311, -0.00565475,  0.00701335,  0.01969632,  0.0088295 ,
        0.0112426 ,  0.00154575, -0.00202277,  0.02458137,  0.01363283,
        0.01919448,  0.00659395,  0.00369342, -0.00307438, -0.00963572,
       -0.01342669,  0.02652991,  0.00328161,  0.01456058,  0.01942626,
        0.00671943,  0.01113122, -0.00713603,  0.01290225, -0.00699026,
       -0.00475354,  0.01233187,  0.01680723,  0.00684344, -0.00567656,
        0.01439512,  0.00313065,  0.01532919,  0.02840651,  0.03631853,
        0.01851448,  0.00754408, -0.01025766, -0.0101532 ,  0.01177904,
        0.00626052, -0.00106759,  0.01077774,  0.00911419,  0.00958991,
        0.03678938,  0.00079037, -0.00210753,  0.00941303, -0.01183442,
        0.00595168,  0.000384  ,  0.01773777,  0.02264878,  0.00659837,
        0.01553729,  0.00539644,  0.00786941, -0.00580284, -0.01261706,
        0.00563437, -0.00046979, -0.00010975,  0.00944363, -0.00076517,
       -0.01532622,  0.02049122,  0.01894525,  0.02118128,  0.00

In [25]:
# isaac과 Jumanji를 내적하는 코드
np.dot(isaac_vector, Jumanji_vector)

0.05261048

isaac이 5점을 준 Jurassic_Park는 선호도가 __0.43541944__가 나왔고, 그외의 영화 Jumanji의 선호도는 __0.05261048__로 isaac이 선호하는 영화에 점수를 더 잘 주었다.

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
---

### 함수 만들기

In [26]:
def get_similar_movie(fav_movie):
    movie_id = movie_to_idx[fav_movie]
    similar_movie = als_model.similar_items(movie_id, N=15)
    #artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommend = [idx_to_movie[i[0]] for i in similar_movie]
    for i in recommend:
        print(movies[movies['movie_id'] == i], '\n')

In [27]:
get_similar_movie(1)

   movie_id             title                        genre
0         1  Toy Story (1995)  Animation|Children's|Comedy 

      movie_id               title                        genre
3045      3114  Toy Story 2 (1999)  Animation|Children's|Comedy 

     movie_id           title                                genre
584       588  Aladdin (1992)  Animation|Children's|Comedy|Musical 

    movie_id        title                    genre
33        34  Babe (1995)  Children's|Comedy|Drama 

      movie_id                 title                        genre
2286      2355  Bug's Life, A (1998)  Animation|Children's|Comedy 

      movie_id                 title           genre
1245      1265  Groundhog Day (1993)  Comedy|Romance 

     movie_id                  title                         genre
360       364  Lion King, The (1994)  Animation|Children's|Musical 

     movie_id                        title                         genre
591       595  Beauty and the Beast (1991)  Animation|Child

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
---

In [28]:
def movie_for_user(user_id):
    user_id = user_to_idx[user_id]
    # recommend에서는 user*item CSR Matrix를 받습니다.
    movie_recommended = als_model.recommend(user_id, csr_data, N=20, filter_already_liked_items=True)
    idx_to_movie = {v:k for k,v in movie_to_idx.items()}
    recommend = [idx_to_movie[i[0]] for i in movie_recommended]
    for i in recommend:
        print(movies[movies['movie_id'] == i], '\n')

In [29]:
movie_for_user(isaac)

     movie_id                    title         genre
898       910  Some Like It Hot (1959)  Comedy|Crime 

      movie_id               title           genre
2992      3061  Holiday Inn (1942)  Comedy|Musical 

      movie_id                   title    genre
3606      3675  White Christmas (1954)  Musical 

      movie_id                   title           genre
3602      3671  Blazing Saddles (1974)  Comedy|Western 

      movie_id                             title               genre
1247      1267  Manchurian Candidate, The (1962)  Film-Noir|Thriller 

     movie_id                                   title  \
590       594  Snow White and the Seven Dwarfs (1937)   

                            genre  
590  Animation|Children's|Musical   

      movie_id                       title                         genre
1019      1032  Alice in Wonderland (1951)  Animation|Children's|Musical 

     movie_id                      title           genre
896       908  North by Northwest (1959)  Dr

`Some Like It Hot (1959)`를 추천해주고 있다. 모델은 왜 `Some Like It Hot (1959)`를 추천해줬을까?`AlternatingLeastSquares` 클래스에 구현된 `explain` 메소드를 사용하면 기록을 남긴 데이터 중 __이 추천에 기여한 정도__를 확인할 수 있다.

## 기여도 확인
---

In [35]:
user = user_to_idx['isaac']

Some_Like_It_Hot = movie_to_idx[910]
explain = als_model.explain(user, csr_data, itemid=Some_Like_It_Hot)

In [37]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[(3114, 0.04779760283296606),
 (1, 0.020718727509404726),
 (1097, -0.0009548447499069159),
 (480, -0.004104011309746896),
 (527, -0.020083074202441598)]

모델이 추천해준 `Some Like It Hot (1959)`는 `comedy` 장르이다. 그래서 그런지 `Toy Story 2 (1999)`와 `Toy Story (1995)`의 기여도가 각각 __0.04779760283296606__, __0.020718727509404726__가 나왔다. 

반면에, `Jurassic Park (1993)`, `E.T. the Extra-Terrestrial (1982)`, `Schindler's List (1993)`는 좀 거리가 있는 장르들이기에 기여도가 각각 __-0.004104011309746896__, __-0.0009548447499069159__, __-0.020083074202441598__가 나왔다고 분석할 수 있다.

## 루브릭
---
1. CSR matrix가 정상적으로 만들어졌다.<br>
(_사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다._)
    
__6040x3628 sparse matrix를 만들었다.__

2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.<br>
(_사용자와 아이템 벡터 내적수치가 의미있게 형성되었다._)
    
__isaac이 선호하는 영화의 내적수치는 0.43541944가 나왔고, <br>
선호하지 않는 영화의 내적수치는 0.05261048로 나왔다.<br>
isaac과 영화의 내적수치가 의미있게 형성된 모습이다.__

3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.<br>
(_MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다._)
    
__유저 선호도나 아이템간 유사도가 잘 나왔고, 기여도에 대해서 분석도 하였다.__

## 회고
---
1. `Toy Story (1995)`의 유사한 영화를 찾았을 때, `Toy Story 2 (1999)`를 추천해주는 것을 보고, 모델을 잘 만들었다고 생각했다.
2. 추천 알고리즘도 자연어에 속할 텐데 음성 프로젝트보다는 난이도가 어렵지 않았다고 느꼈다. 
3. 기여도 분석 단계에서 노드의 설명이 부족하여 좀 헷갈렸으나, slack과 구글링을 통해 기여도 분석도 잘 마무리 했다.